Primero importamos las librerias necesarias

In [27]:
import tweepy,json
import pandas as pd
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Stream
from tweepy.streaming import StreamListener
import time
import sys

Abrimos el archivo JSON que contiene los tweets, en el nombre del archivo se indica la fecha y la hora en la que es recogido

In [28]:
tweets = []
for line in open('streamer_20190216-233316.json', 'r'):
    if line.strip():
        tweets.append(json.loads(line))



Pasamos los tweets a dataframe, las columnas que vamos a utilizar dentro de 'user' son las siguientes:
* id
* name
* screen_name
* location
* verified
* followers_count
* friends_count
* listed_count
* favourites_count
* statuses_count
* created_at
* utc_offset
* time_zone
* geo_enabled
* lang
* contributors_enabled
* is_translator
* following
* follow_request_sent
* notifications

In [31]:
df = pd.DataFrame(tweets)
df['user'][0]

{'id': 899100235028459521,
 'id_str': '899100235028459521',
 'name': 'C.Drexler',
 'screen_name': 'ClydeDrexlerB',
 'location': None,
 'url': None,
 'description': None,
 'translator_type': 'none',
 'protected': False,
 'verified': False,
 'followers_count': 149,
 'friends_count': 546,
 'listed_count': 1,
 'favourites_count': 5221,
 'statuses_count': 5331,
 'created_at': 'Sun Aug 20 02:46:15 +0000 2017',
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': False,
 'lang': 'es',
 'contributors_enabled': False,
 'is_translator': False,
 'profile_background_color': 'F5F8FA',
 'profile_background_image_url': '',
 'profile_background_image_url_https': '',
 'profile_background_tile': False,
 'profile_link_color': '1DA1F2',
 'profile_sidebar_border_color': 'C0DEED',
 'profile_sidebar_fill_color': 'DDEEF6',
 'profile_text_color': '333333',
 'profile_use_background_image': True,
 'profile_image_url': 'http://pbs.twimg.com/profile_images/971508835318984706/tEwrcGEz_normal.jpg',
 'profile_ima

In [33]:
len(tweets)

9659

In [4]:
lista_entendidos = []
for tweet in tweets:
    if 'extended_tweet' in tweet:
        lista_entendidos.append(tweet)
len(lista_entendidos)/len(tweets)*100

14.389291689905187

In [26]:
lista_influencers = []
for tweet in tweets:
    lista_influencers.append([tweet['user']['name'],tweet['user']['followers_count']])
    

sorted(lista_influencers)

KeyError: 'user'

In [5]:
diccionario_tweets = {}
for x in tweets: 
    for k,v in tweets[x].items():
        diccionario_tweets[k] = v
diccionario_tweets


TypeError: list indices must be integers or slices, not dict

In [16]:
def flatten_tweets(tweets_json):
    """ Flattens out tweet dictionaries so relevant JSON
        is in a top-level dictionary."""
    tweets_list = []
    
    # Iterate through each tweet
    for tweet in tweets_json:
        tweet_obj = json.loads(tweet)
    
        # Store the user screen name in 'user-screen_name'
        tweet_obj['user-screen_name'] = tweet_obj['user']['screen_name']
    
        # Check if this is a 140+ character tweet
        if 'extended_tweet' in tweet_obj:
            # Store the extended tweet text in 'extended_tweet-full_text'
            tweet_obj['extended_tweet-full_text'] = tweet_obj['extended_tweet']['full_text']
    
        if 'retweeted_status' in tweet_obj:
            # Store the retweet user screen name in 'retweeted_status-user-screen_name'
            tweet_obj['retweeted_status-user-screen_name'] = tweet_obj['retweeted_status']['user']['screen_name']

            # Store the retweet text in 'retweeted_status-text'
            tweet_obj['retweeted_status-text'] = tweet_obj['retweeted_status']['text']
            
        tweets_list.append(tweet_obj)
    return tweets_list

In [6]:
df = pd.DataFrame(tweets)
df.columns

Index(['contributors', 'coordinates', 'created_at', 'display_text_range',
       'entities', 'extended_entities', 'extended_tweet', 'favorite_count',
       'favorited', 'filter_level', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place',
       'possibly_sensitive', 'quote_count', 'quoted_status',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink',
       'reply_count', 'retweet_count', 'retweeted', 'retweeted_status',
       'source', 'text', 'timestamp_ms', 'truncated', 'user'],
      dtype='object')